In [1]:
import glob
import os
import pandas as pd
import numpy as np
import pyproj
from tqdm import tqdm
from multiprocessing import Pool
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_excel('../外部資料集/police_office.ods', engine='odf')
df = df.drop_duplicates(subset=['地址'], keep='first')
df = df.reset_index()
df

,index,縣市,中文單位名稱,單位,英文單位名稱,郵遞區號,地址,電話,POINT_X,POINT_Y,英文地址
0,0,臺北市,臺北市政府警察局,NaN,Taipei City Police Department,100,臺北市中正區延平南路96號,02 23313561,301442.646800,2.770734e+06,"No. 96, Yanping S. Rd., Zhongzheng Dist., Taip..."
1,1,臺北市,中山分局,分局,Zhongshan Police Precinct,104,臺北市中山區中山北路2段1號,02 25412491,302750.697400,2.771704e+06,"No. 1, Sec. 2, Zhongshan N. Rd., Zhongshan Dis..."
2,2,臺北市,中山一派出所,派出所,Zhongshan First Police Station,104,臺北市中山區中山北路1段110號,02 25519321,302613.545700,2.771416e+06,"No. 110, Sec. 1, Zhongshan N. Rd., Zhongshan D..."
3,3,臺北市,中山二派出所,派出所,Zhongshan Second Police Station,104,臺北市中山區中山北路2段59巷21號,02 25517771,25.055917,1.215239e+02,"No. 21, Ln59, Sec. 2, Zhongshan N. Rd., Zhongs..."
4,4,臺北市,圓山派出所,派出所,Yuanshan Police Staticn,104,臺北市中山區中山北路3段62號,02 25969433,302834.983800,2.773934e+06,"No. 62, Sec. 3, Zhongshan N. Rd., Zhongshan Di..."
...,...,...,...,...,...,...,...,...,...,...,...
1551,1676,連江縣,北竿警察所,警察所,Beigan Police Station,210,連江縣北竿鄉塘岐村259號,083 655234,149784.680600,2.901895e+06,"No. 259, Tangqi Vil., Beigan Township, Lienchi..."
1552,1677,連江縣,莒光警察所,警察所,Juguang Police Station,211,連江縣莒光鄉田沃村66之4號,083 688153,143869.320800,2.874258e+06,"No. 66-4, Tianwo Vil., Juguang Township, Lienc..."
1553,1678,連江縣,東莒派出所,派出所,Dongjyu Police Station,211,連江縣莒光鄉大坪村81-1號,083 689046,146976.060000,2.872480e+06,"No. 81-1, Daping Vil., Juguang Township, Lienc..."
1554,1679,連江縣,東引警察所,警察所,Dongyin Police Station,212,連江縣東引鄉中柳村121之1號,083 677204,199074.510900,2.917380e+06,"No. 121-1, Zhongliu Vil., Dongyin Township, Li..."


In [3]:
twd97 = pyproj.Proj(init='epsg:3826')  # TWD97
wgs84 = pyproj.Proj(init='epsg:4326')  # WGS84

In [4]:
def get_coordinate(row):
    lon, lat = pyproj.transform(twd97, wgs84, row['POINT_X'], row['POINT_Y'])
    return {
        'index': row['index'],
        'lon': lon,
        'lat': lat
    }

with Pool(22) as pool:
    coordiates = list(tqdm(pool.imap(get_coordinate, df.to_dict('records')), total=len(df)))
df_coordiates = pd.DataFrame(coordiates).fillna(-999999)
df = df.merge(df_coordiates, how='left', on='index')
df

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1556/1556 [00:02<00:00, 695.24it/s]


,index,縣市,中文單位名稱,單位,英文單位名稱,郵遞區號,地址,電話,POINT_X,POINT_Y,英文地址,lon,lat
0,0,臺北市,臺北市政府警察局,NaN,Taipei City Police Department,100,臺北市中正區延平南路96號,02 23313561,301442.646800,2.770734e+06,"No. 96, Yanping S. Rd., Zhongzheng Dist., Taip...",121.509813,25.043871
1,1,臺北市,中山分局,分局,Zhongshan Police Precinct,104,臺北市中山區中山北路2段1號,02 25412491,302750.697400,2.771704e+06,"No. 1, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",121.522813,25.052590
2,2,臺北市,中山一派出所,派出所,Zhongshan First Police Station,104,臺北市中山區中山北路1段110號,02 25519321,302613.545700,2.771416e+06,"No. 110, Sec. 1, Zhongshan N. Rd., Zhongshan D...",121.521442,25.049989
3,3,臺北市,中山二派出所,派出所,Zhongshan Second Police Station,104,臺北市中山區中山北路2段59巷21號,02 25517771,25.055917,1.215239e+02,"No. 21, Ln59, Sec. 2, Zhongshan N. Rd., Zhongs...",118.754791,0.001098
4,4,臺北市,圓山派出所,派出所,Yuanshan Police Staticn,104,臺北市中山區中山北路3段62號,02 25969433,302834.983800,2.773934e+06,"No. 62, Sec. 3, Zhongshan N. Rd., Zhongshan Di...",121.523734,25.072715
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,1676,連江縣,北竿警察所,警察所,Beigan Police Station,210,連江縣北竿鄉塘岐村259號,083 655234,149784.680600,2.901895e+06,"No. 259, Tangqi Vil., Beigan Township, Lienchi...",119.997035,26.225321
1552,1677,連江縣,莒光警察所,警察所,Juguang Police Station,211,連江縣莒光鄉田沃村66之4號,083 688153,143869.320800,2.874258e+06,"No. 66-4, Tianwo Vil., Juguang Township, Lienc...",119.940093,25.975456
1553,1678,連江縣,東莒派出所,派出所,Dongjyu Police Station,211,連江縣莒光鄉大坪村81-1號,083 689046,146976.060000,2.872480e+06,"No. 81-1, Daping Vil., Juguang Township, Lienc...",119.971255,25.959635
1554,1679,連江縣,東引警察所,警察所,Dongyin Police Station,212,連江縣東引鄉中柳村121之1號,083 677204,199074.510900,2.917380e+06,"No. 121-1, Zhongliu Vil., Dongyin Township, Li...",120.489698,26.367687


In [5]:
df.rename(columns={
    '中文單位名稱': 'name',
    '地址': 'address'
}, inplace=True)
use_cols = ['name', 'address', 'lat', 'lon']
df[use_cols].to_csv('../官方資料集/external_data/police_office.csv', index=False, encoding='utf-8')

In [6]:
df[use_cols].describe()

,lat,lon
count,1556.000000,1556.000000
mean,23.863446,120.870622
std,1.724701,0.754009
min,0.001095,118.242686
25%,23.190059,120.438854
50%,23.982608,120.747001
75%,24.827312,121.395683
max,26.367687,140.191232
